### Use the BookOps WorldCat wrapper to pull OCLC numbers based on a list of ISBNs

##### Import libraries
This section loads the packages needed to work with data, send API requests, and use the BookOps wrapper with OCLC's WorldCat API.

In [ ]:
import pandas as pd
import requests
from bookops_worldcat import WorldcatAccessToken
import time
import re

##### Configure access token
This section contains the authentication details required by the WorldCat API. 'mykey' and 'mysecret' should be updated based on the user's credentials.

In [ ]:
#Configure access token
WORLDCAT_KEY = 'mykey'
WORLDCAT_SECRET = 'mysecret'
SCOPES = 'WorldCatMetadataAPI'

##### Configure files
This section contains the filepath and name of the file that will be read (INPUT_FILE) and the file where results will be saved (OUTPUT_FILE).

In [ ]:
#Configure files
INPUT_FILE = 'FILENAME'
OUTPUT_FILE = 'FILENAME.xlsx'

##### Generate an access token
The **get_token** function uses the API credentials specified above to create a token to access the WorldCat API. Tokens expire after twenty minutes, and should automatically refresh within the script.

In [ ]:
#Generate an access token
def get_token():
    return WorldcatAccessToken(
        key=WORLDCAT_KEY,
        secret=WORLDCAT_SECRET,
        scopes=SCOPES
    )

##### Get OCLC numbers

The **get_oclc_numbers** function takes an ISBN from INPUT_FILE, queries the WorldCat API, and returns the first OCLC number associated with the ISBN.

In [ ]:
#Get OCLC numbers
def get_oclc_numbers(isbn, token):
    try:
        url = f'https://americas.discovery.api.oclc.org/worldcat/search/v2/brief-bibs'
        headers = {
            'Authorization': f'Bearer {token.token_str}',
            'Accept': 'application/json'
        }
        params = {'q': f'bn:{isbn}'}

        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            return data["briefRecords"][0]["oclcNumber"]
        
        except (KeyError, IndexError, TypeError):
            return {}
        
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch data for ISBN {isbn}: {e}")
        return {}

##### Run the workflow

The **main** function performs the following steps:
1. Reads INPUT_FILE
2. Creates an API token
3. Runs through each ISBN and sends a query to get a corresponding OCLC number
4. Collects the results of the queries and merges them back with the original data
5. Adds a RECORD_ID column with a unique ID for each record
6. Exports the final dataset as an Excel file

Depending on the structure of INPUT_FILE, names of fields may need to be updated. For example, the file structure here uses "e-isbn" as a field name. A different file may use "online_identifier" instead, which means either the script below needs to be updated to use "online_identifier", or INPUT_FILE needs to be updated to use "e-isbn".

In [ ]:
#Run query and export results
def main():
    isbnlist_df = pd.read_excel(INPUT_FILE, dtype={'publication_title': str, 'e-isbn': str})

    oclc_numbers = []

    token = get_token()

    for isbn in isbnlist_df['e-isbn']:
        isbn = str(isbn).strip()
        if not isbn:
            continue

        if token.is_expired():
            print("Refreshing token!")
            token = get_token()

        result = get_oclc_numbers(isbn, token)

        oclc_number = result if result and result != "None" else "None"

        oclc_numbers.append({"e-isbn": isbn, "OCLC_NUMBER": oclc_number})

        print(f"{isbn}, {result}")
        time.sleep(0.2)

    oclc_numbers = pd.DataFrame(oclc_numbers)

    final_df = isbnlist_df.merge(oclc_numbers, on='e-isbn', how='left')
    final_df.insert(0, 'RECORD_ID', range(1, len(final_df) + 1))
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"Data exported to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

##### Next steps
OCLC numbers can be used in the below workflows:
1. Get other editions - OCLCGetOtherEditions.ipynb
2. Get LC classification data - OCLC_GetLCData.ipynb
3. Check library holdings/total OCLC holdings - OCLC_CheckLibraryHoldings.ipynb